# Fix positions

This notebook fixes positions in hg38 for Missense protein variant types. 

In [1]:
import pandas as pd
from Bio import SeqIO

In [2]:
import helpers

In [3]:
database_folder = '../intermediate_pipeline_db_versions'

In [4]:
input_path_prefix = f'{database_folder}/00010_2024-01-18-20-05-26-311470_Human-TAFAZZIN-Variants-Database_' 
# TODO get this automatically in next notebooks

output_path_prefix = helpers.create_database_output_path_prefix()

version number: 00020
database output path prefix: ../intermediate_pipeline_db_versions/00020_2024-01-24-18-35-54-354423_Human-TAFAZZIN-Variants-Database_


In [5]:
taz_sequences_path = '../data_external/Homo_sapiens_ENST00000601016_6_sequence.fa'
taz_positions_path = '../data_external/ExonsSpreadsheet-Homo_sapiens_Transcript_Exons_ENST00000601016_whole_table.csv'

### Load positions

TAZ positions from ensembl.org in hg38. 

In [6]:
taz_positions = pd.read_csv(taz_positions_path)

In [7]:
taz_positions['Start'] = pd.to_numeric(taz_positions['Start'].str.replace(',', ''), errors='coerce')
taz_positions['Start'] = taz_positions['Start'].fillna(0).astype(int)

In [8]:
taz_positions['End'] = pd.to_numeric(taz_positions['End'].str.replace(',', ''), errors='coerce')
taz_positions['End'] = taz_positions['End'].fillna(0).astype(int)

In [9]:
taz_positions

,No.,Exon / Intron,Start,End,Start Phase,End Phase,Length,Sequence
0,NaN,5' upstream sequence,0,0,NaN,NaN,NaN,"{""provisional"":{..."
1,1,ENSE00003022330,154411539,154411952,-,1,414,"{""url"":""/Homo_sa..."
2,,Intron 1-2,154411953,154412085,NaN,NaN,133,"{""provisional"":{..."
3,2,ENSE00003127263,154412086,154412214,1,1,129,"{""provisional"":{..."
4,,Intron 2-3,154412215,154413206,NaN,NaN,992,"{""provisional"":{..."
5,3,ENSE00003736420,154413207,154413252,1,2,46,"{""url"":""/Homo_sa..."
6,,Intron 3-4,154413253,154413481,NaN,NaN,229,"{""provisional"":{..."
7,4,ENSE00003017673,154413482,154413567,2,1,86,"{""provisional"":{..."
8,,Intron 4-5,154413568,154414100,NaN,NaN,533,"{""url"":""/Homo_sa..."
9,5,ENSE00003050674,154414101,154414190,1,1,90,"{""provisional"":{..."


#### Sanity check for length of exon:

In [10]:
taz_positions_exons_only = taz_positions[(taz_positions['No.'] != ' ') & (~taz_positions['No.'].isna())].copy(deep=True)
taz_positions_exons_only

,No.,Exon / Intron,Start,End,Start Phase,End Phase,Length,Sequence
1,1,ENSE00003022330,154411539,154411952,-,1,414,"{""url"":""/Homo_sa..."
3,2,ENSE00003127263,154412086,154412214,1,1,129,"{""provisional"":{..."
5,3,ENSE00003736420,154413207,154413252,1,2,46,"{""url"":""/Homo_sa..."
7,4,ENSE00003017673,154413482,154413567,2,1,86,"{""provisional"":{..."
9,5,ENSE00003050674,154414101,154414190,1,1,90,"{""provisional"":{..."
11,6,ENSE00003154518,154419543,154419623,1,1,81,"{""provisional"":{..."
13,7,ENSE00003158525,154419705,154419746,1,1,42,"{""url"":""/Homo_sa..."
15,8,ENSE00003787276,154420032,154420094,1,1,63,"{""url"":""/Homo_sa..."
17,9,ENSE00003724812,154420212,154420264,1,0,53,"{""provisional"":{..."
19,10,ENSE00003736683,154420658,154420735,0,0,78,"{""url"":""/Homo_sa..."


In [11]:
taz_positions_exons_only['Length'] = taz_positions_exons_only['Length'].astype(int)

In [12]:
taz_positions_exons_only['Length'].sum()

1906

### Load TAZ sequences

from ensembl.org

In [13]:
for seq_record in SeqIO.parse(taz_sequences_path, "fasta"):
    print(seq_record.description)
    print(len(seq_record))
    print('-----')

ENST00000601016.6 TAFAZZIN-213 cdna:protein_coding
1906
-----
TAFAZZIN-213 cds:protein_coding
879
-----
TAFAZZIN-213 ENSE00003022330 exon:protein_coding
414
-----
TAFAZZIN-213 ENSE00003127263 exon:protein_coding
129
-----
TAFAZZIN-213 ENSE00003736420 exon:protein_coding
46
-----
TAFAZZIN-213 ENSE00003017673 exon:protein_coding
86
-----
TAFAZZIN-213 ENSE00003050674 exon:protein_coding
90
-----
TAFAZZIN-213 ENSE00003154518 exon:protein_coding
81
-----
TAFAZZIN-213 ENSE00003158525 exon:protein_coding
42
-----
TAFAZZIN-213 ENSE00003787276 exon:protein_coding
63
-----
TAFAZZIN-213 ENSE00003724812 exon:protein_coding
53
-----
TAFAZZIN-213 ENSE00003736683 exon:protein_coding
78
-----
TAFAZZIN-213 ENSE00003627163 exon:protein_coding
824
-----
TAFAZZIN-213 intron 1:protein_coding
133
-----
TAFAZZIN-213 intron 2:protein_coding
992
-----
TAFAZZIN-213 intron 3:protein_coding
229
-----
TAFAZZIN-213 intron 4:protein_coding
533
-----
TAFAZZIN-213 intron 5:protein_coding
5352
-----
TAFAZZIN-213 intron

In [14]:
len(seq_record)

10188

In [15]:
seq_record.seq

Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...AAA')

In [16]:
seq_record.seq[-1]

'A'

In [17]:
seq_record

SeqRecord(seq=Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...AAA'), id='X', name='X', description='X dna:chromosome chromosome:GRCh38:X:154411539:154421726:1', dbxrefs=[])

### Load Tafazzin database

In [18]:
df_pathogenic = pd.read_csv(input_path_prefix + 'pathogenic.csv')
df_vus = pd.read_csv(input_path_prefix + 'vus.csv')
df_exon5 = pd.read_csv(input_path_prefix + 'exon5.csv')
df_benign = pd.read_csv(input_path_prefix + 'benign.csv')

In [19]:
df_pathogenic.head(1)

,Location,Location in Genome release 37 (hg19),Location in Genome release 38 (hg38),Protein Variant Type,Impact of Variant,DNA Modifications,Protein or mRNA Variants,Functional outcome (MLCL/CL ratio),Taffazin Functional motifs,Method of Validation,References,Source,Additional variants in other genes,Location and Order of Discovery,Notes,Unnamed: 15
0,Exon 1,X:153640189,X:154411852,Frameshift,NaN,c.9_10dupG,p.His4Alafs*130,MLCL/CL elevated,NaN,NaN,Ref. 1 (Pat.1); Ref. 80; Ref. 113,NaN,NaN,1-1,NaN,NaN


In [20]:
loc_38_col = 'Location in Genome release 38 (hg38)'
loc_37_col = 'Location in Genome release 37 (hg19)'

In [21]:
def get_row_from_taz_positions(taz_database_38_loc, taz_positions=taz_positions):
    # get row for taz_database_38_loc with exon/intron info. 
    matching_rows = taz_positions[(taz_positions['Start'] <= taz_database_38_loc) & (taz_positions['End'] >= taz_database_38_loc)]
    # If you expect only one match and want to get a single value
    if len(matching_rows) == 1:
        return matching_rows.iloc[0]
    else:
        print(f"Multiple or no matching rows found! {taz_database_38_loc}")
        return None

In [22]:
def find_seq_record(exon_intron_info, taz_sequences_path=taz_sequences_path):
    # Find sequence from exon / intron info
    # Use a generator expression to find the first matching seq_record
    seq_record = next((record for record in SeqIO.parse(taz_sequences_path, "fasta") if exon_intron_info in record.description), None)
    return seq_record


# Find unexpected letters on position, based on hg38.

We do not have Protein Variant type information in other sheets than Pathogenic, so we just focus on that & missense. 

In [23]:
weird_splits = []
for key, row in df_pathogenic.iterrows():
    try:
        protein_variant_type = row['Protein Variant Type']
        if not(protein_variant_type == 'Missense'): 
            continue
            
        taz_database_38_loc = int(row[loc_38_col].split(':')[1])
        taz_database_37_loc = int(row[loc_37_col].split(':')[1])
        dna_modifications = row['DNA Modifications']
        
        row_from_taz_positions = get_row_from_taz_positions(taz_database_38_loc)
        exon_intron_from_taz_positions = row_from_taz_positions['Exon / Intron']
        start_from_taz_positions = row_from_taz_positions['Start']
        end_from_taz_positions = row_from_taz_positions['End']
        seq_from_taz_positions = row_from_taz_positions.Sequence.split('}')[-1].strip()
        
        seq_record = find_seq_record(exon_intron_from_taz_positions)

        if seq_record:
            sequence = seq_record.seq
            pos_in_sequence = taz_database_38_loc - start_from_taz_positions
            expected_letter = sequence[pos_in_sequence]
            
            dna_modifications_split = dna_modifications.split('>')
            assert len(dna_modifications_split) >= 2, f'problem with split for {dna_modifications}; {dna_modifications_split}'
            
            if len(dna_modifications_split) > 2:
                weird_splits.append(f'weird split for "{dna_modifications}"; {dna_modifications_split}; {taz_database_38_loc}')
            
            dna_modifications_letter = dna_modifications_split[0].strip()[-1]
            if dna_modifications_letter != expected_letter:
                print(f'Expected from position {taz_database_38_loc}: {expected_letter} ; dna modifications: {dna_modifications}')
                #display(row)
                #print('=======================')
            # TODO hg37, corner cases??
        else:
            print("No matching record found.")
        
    except Exception as ex:
        print('Exception', ex, key, row[loc_38_col])
        

Expected from position 154412187: T ; dna modifications: c.227C>G
Expected from position 154413499: A ; dna modifications: c.302G>T   
Expected from position 154420086: G ; dna modifications: c.640C>T  


# TODO report these ^^

In [24]:
for weird_split in weird_splits:
    print(weird_split)

weird split for "c.590G>A    and   c.791A>G  "; ['c.590G', 'A    and   c.791A', 'G  ']; 154420038
weird split for "c.602C>T  in NM_000116     (publ as c.469 C>T, p.Leu157Phe, in NM_181313)"; ['c.602C', 'T  in NM_000116     (publ as c.469 C', 'T, p.Leu157Phe, in NM_181313)']; 154420050
weird split for "c.646G>A/C GGA>A/CGA"; ['c.646G', 'A/C GGA', 'A/CGA']; 154420094


In [25]:
df_pathogenic['Protein Variant Type'].value_counts()

Missense               173
Frameshift              73
Splicing                54
Nonsense                48
Deletion                36
Splicing/Silent          7
Deletion-Insertion       3
Deletion (in frame)      2
Splicing/Insertion       2
Silent                   1
Insertion-Deletion       1
Splicing/Frameshift      1
Insertion                1
Splicing/Deletion        1
Name: Protein Variant Type, dtype: int64

In [26]:
df_pathogenic['Protein Variant Type'].value_counts(normalize=True)

Missense               0.429280
Frameshift             0.181141
Splicing               0.133995
Nonsense               0.119107
Deletion               0.089330
Splicing/Silent        0.017370
Deletion-Insertion     0.007444
Deletion (in frame)    0.004963
Splicing/Insertion     0.004963
Silent                 0.002481
Insertion-Deletion     0.002481
Splicing/Frameshift    0.002481
Insertion              0.002481
Splicing/Deletion      0.002481
Name: Protein Variant Type, dtype: float64

In [29]:
df_pathogenic[df_pathogenic['Protein Variant Type'] == 'Missense']['Location in Genome release 38 (hg38)'].nunique()

58

^^ TODO 

[x] check cases when pathogenic mutations are not in Alpha missense --- are these mistakes in Tafazzin database? -- yes, fix & report this..
* should we try to fix also other most frequent, such as frameshift, splicing, nonsense, deletion?
* does fixing hg37 make sense? or is it obsolete anyway?
* what about other sheets, where we do not have protein variant type info? can we automatically get it? (task for chatgpt?)
* can other tasks from TODO be solved with chatgpt?


https://github.com/orgs/community/discussions/63296